# Automated SEC Keyword Scraping Tool

This notebook automatically searched through the SEC filings of companies and returns the number of times a keyword appears in each of the companies filings. It also looks into whether that keyword is used in a table and returns a data frame of the how the word is used and how many times each keyword appears in a table. Currently it is set up to search for the adoption of Cryptocurrency related keywords.

## Import Packages

In [1]:
import edgar
import pandas as pd
import numpy as np
import time
import os
from bs4 import BeautifulSoup
import bs4 as bs
import codecs
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import random
import urllib.request
from collections import OrderedDict
import json
import re
from datetime import datetime
from datetime import date
from urllib.request import urlopen
start_time = datetime.now()

In [2]:
headers_list = [
    # Firefox 77 Mac
    {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Firefox 77 Windows
    {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 83 Mac
    {
    "Connection": "keep-alive",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
    # Chrome 83 Windows 
    {
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9"
    }
]

## This will import every CIK number for public companies

In [3]:
cik_url="https://www.sec.gov/files/company_tickers.json"
response = urlopen(cik_url)
data_json = json.loads(response.read())
data_cik=pd.DataFrame(data_json).T
CIK = data_cik['cik_str']

In [4]:
#Remove Duplicates
CIK = list(dict.fromkeys(CIK))

In [5]:
data_cik

,cik_str,ticker,title
0,320193,AAPL,Apple Inc.
1,789019,MSFT,MICROSOFT CORP
2,1652044,GOOGL,Alphabet Inc.
3,1018724,AMZN,AMAZON COM INC
4,1318605,TSLA,"Tesla, Inc."
...,...,...,...
12206,1857410,IVCBW,Investcorp Europe Acquisition Corp I
12207,1858912,GDNRU,Gardiner Healthcare Acquisitions Corp.
12208,1860482,TOACW,Talon 1 Acquisition Corp
12209,1860484,RACYW,Relativity Acquisition Corp


For illustration purposes I have selected a few CIK Numbers below as running it for every company takes a while. In order to run it for every company comment out the cell below with a #.

In [6]:
CIK = ['896493','1507605','1403161' '19617', '1318605', '0001457512', '0001702014', '0001709702', '0001050446','0001167419' ]

## Extract URLs

In [7]:
filling_type = '10-K' #8-K 10-Q
num_of_fillings = 2 #Number of filings for each company
within_year = 2 #Number of years to go back
recent_year = date.today().year

name_lst = []
cik_lst = []
sic_lst = []
industry_type_lst = []
state_location_lst = []
state_inc_lst = []
filling_date_lst = []
url_lst = []



for item in CIK:
    link = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=' + str(item) + '&type=' + filling_type + '&dateb=&count=100&scd=filings'
    print(link) #Use for debug, if you find the bug, click the url and check why is the bug
    headers = random.choice(headers_list)
    r = requests.Session()
    r.headers = headers
    retry = Retry(connect=5, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    r.mount('http://', adapter)
    r.mount('https://', adapter)
    #time.sleep(np.random.uniform(1,2))
    try:
        html = r.get(link).text
    except TimeoutError:
        continue
    soup = BeautifulSoup(html, "html.parser")
    
    if soup.find("div", class_="companyInfo") != None: #If we can find company info table
        if soup.find('span', class_ = 'companyName') != None: #If we can find company name and CIK
            company_info = str(soup.find("span", class_ = 'companyName'))
            start = company_info.find('>')
            end = company_info.find('<acronym')
            company_name = company_info[start+1:end].replace('amp;', '').replace('  ', ' ')
            company_cik = re.findall(r"CIK=(\d+)", company_info)[0]
            
            if soup.find('p', class_ = 'identInfo') != None: #If we can find company identification info
                company_ident = soup.find('p', class_ = 'identInfo')
                if company_ident.find('acronym', title="Standard Industrial Code") != None: #if we can find SIC code
                    company_ident = str(company_ident)
                    company_sic = re.findall(r"SIC=(\d+)", company_ident)[0]
                    if re.findall(r"- ([&;\w\sA-Za-z]+)", company_ident) != []: #if we can find company industry
                        company_industry = re.findall(r"- ([&;\w\sA-Za-z]+)", company_ident)[0].replace('amp;', '')
                        if (company_ident.find('State=') != -1) & (company_ident.find('State of Inc.') != -1): # if we can find state location and state of Inc.
                            company_state_location = re.findall(r"State=([a-zA-Z0-9]+)", company_ident)[0]
                            company_state_inc = re.findall(r"<strong>([a-zA-Z0-9]+)</strong>", company_ident)[0]
                            if soup.find('table', class_ = 'tableFile2') != None: #if we can find filling table
                                table = soup.find('table', class_ = 'tableFile2')
                                fillings = table.find_all('tr')[1:] #find all documents
                            
                                for filling in fillings: #search each filling
                                    document_info = filling.find_all('td')
                                    if (document_info[0].string == filling_type) and (int(document_info[3].string[:4]) > (recent_year - within_year)): #if we the document is 10-K and the year of filling within range and dont reach search limit
                                        filling_date = document_info[3].string
                                        url = "https://www.sec.gov" + str(document_info[1].find('a', id = 'documentsbutton', href = True))[9:-37]
                                        html = r.get(url).text
                                        html = html[html.find('td scope="row">1<'):]
                                        html = html[html.find('/Archives/edgar/data/'):]
                                        filling_url = "https://www.sec.gov" + html[0:html.find('>') - 1] #Find the 10-K url
                                        name_lst.append(company_name)
                                        cik_lst.append(company_cik)
                                        sic_lst.append(company_sic)
                                        industry_type_lst.append(company_industry)
                                        state_location_lst.append(company_state_location)
                                        state_inc_lst.append(company_state_inc)
                                        filling_date_lst.append(filling_date)
                                        url_lst.append(filling_url)
                                    
                                    else:
                                        continue
                            else:#if we cannot find filling table
                                continue
                        else: #if we cannot find company state and state of inc
                            continue
                    else: # if we cannot find company industry
                        continue
                else: #if we cannot find SIC code
                    continue
            else: #If we cannot find company identification info
                continue
        else: #If we cannot find the company name and CIK
            continue
    else: #If we cannot find company info
        continue    

https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=896493&type=10-K&dateb=&count=100&scd=filings
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1507605&type=10-K&dateb=&count=100&scd=filings
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=140316119617&type=10-K&dateb=&count=100&scd=filings
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1318605&type=10-K&dateb=&count=100&scd=filings
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001457512&type=10-K&dateb=&count=100&scd=filings
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001702014&type=10-K&dateb=&count=100&scd=filings
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001709702&type=10-K&dateb=&count=100&scd=filings
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001050446&type=10-K&dateb=&count=100&scd=filings
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001167419&type=10-K&dateb=&count=100&scd

In [8]:
company_df = pd.DataFrame({'CIK':cik_lst, 'Company Name':name_lst, 'SIC':sic_lst, 
                           'Industry':industry_type_lst, 'State Location':state_location_lst, 
                           'State Inc':state_inc_lst, 'Filling Date':filling_date_lst,
                          'URL':url_lst})

company_df = company_df.groupby('CIK').head(num_of_fillings) #Choose the top n (hyperparameter can be set by client) fillings

## Input Keywords to Search For

In [9]:
keywords = ['digital asset','cryptocurrency', 'cryptocurrencies', 'digital holdings','bitcoin','ethereum']

In [10]:
keywords_table_list=[]
for key in keywords:
    keywords_table = key + '_table'
    keywords_table_list.append(keywords_table)

In [11]:
columns = ['URL'] + keywords + keywords_table_list

## Search Through URLs for Keywords

In [12]:

data = pd.DataFrame(columns = columns)
whole_phrase ={}

for url in company_df['URL']:
    print('RUN TIME: ',datetime.now() - start_time)
    headers = random.choice(headers_list)
    #time.sleep(np.random.uniform(1,2))
    try:
        html = requests.get(url, headers=headers, timeout=(10,200)).text.lower() #, verify=False
        
    except:
        continue
        print('Timeout has been raised.')
        
    keyword_lst=[]
    keyword_table_lst=[]
    list_phrases=[]
    for key in keywords:
        keyword_lst.append(html.count(key))
        
        
        keyword_pos_lst = [pos.start() for pos in re.finditer(key, html)]
        keyword_cnt_table = 0
        for keyword_pos in keyword_pos_lst:
            if '</td>' in html[keyword_pos:keyword_pos + 50]:
                keyword_cnt_table += 1
                section = html[keyword_pos-50:keyword_pos + 50]
                start = '>'
                end = '<'
                phrase = (section.split(start))[1].split(end)[0]
                if phrase:
                    list_phrases.append(phrase)
                
                
            else:
                continue
        keyword_table_lst.append(keyword_cnt_table)
    keywords_all=[]    
    keywords_all = [url] + keyword_lst + keyword_table_lst                          
        
    df_length = len(data)
    data.loc[df_length] = keywords_all
    
    if len(list_phrases)>0:
        whole_phrase[url]=list_phrases
    


RUN TIME:  0:00:02.353455
RUN TIME:  0:00:02.575957
RUN TIME:  0:00:02.741547
RUN TIME:  0:00:02.906529
RUN TIME:  0:00:03.060986
RUN TIME:  0:00:03.348207
RUN TIME:  0:00:03.647213
RUN TIME:  0:00:03.862915
RUN TIME:  0:00:04.080198
RUN TIME:  0:00:04.224803


In [13]:
data_location_table=pd.DataFrame.from_dict(whole_phrase, orient='index')
data_location_table = data_location_table.reset_index()
data_location_table.rename(columns = {'index':'URL'}, inplace = True)
results_location_table = company_df.merge(data_location_table,on='URL')

## Results for usage of keyword within table

In [14]:
results_location_table.T.head(10)

,0,1,2,3,4,5,6,7,8
CIK,0000896493,0000896493,0001507605,0001507605,0001318605,0001050446,0001050446,0001167419,0001167419
Company Name,"BitNile Holdings, Inc.","BitNile Holdings, Inc.","MARATHON DIGITAL HOLDINGS, INC.","MARATHON DIGITAL HOLDINGS, INC.","Tesla, Inc.",MICROSTRATEGY Inc,MICROSTRATEGY Inc,"Riot Blockchain, Inc.","Riot Blockchain, Inc."
SIC,3679,3679,7374,7374,3711,7372,7372,7374,7374
Industry,ELECTRONIC COMPONENTS,ELECTRONIC COMPONENTS,SERVICES,SERVICES,MOTOR VEHICLES & PASSENGER CAR BODIES,SERVICES,SERVICES,SERVICES,SERVICES
State Location,NV,NV,NV,NV,CA,VA,VA,CO,CO
State Inc,DE,DE,NV,NV,DE,DE,DE,NV,NV
Filling Date,2022-04-15,2021-04-15,2022-03-10,2021-03-16,2022-02-07,2022-02-16,2021-02-12,2022-03-16,2021-03-31
URL,https://www.sec.gov/Archives/edgar/data/896493...,https://www.sec.gov/Archives/edgar/data/896493...,https://www.sec.gov/Archives/edgar/data/150760...,https://www.sec.gov/Archives/edgar/data/150760...,https://www.sec.gov/Archives/edgar/data/131860...,https://www.sec.gov/Archives/edgar/data/105044...,https://www.sec.gov/Archives/edgar/data/105044...,https://www.sec.gov/Archives/edgar/data/116741...,https://www.sec.gov/Archives/edgar/data/116741...
0,\n,\n,impairment of mined cryptocurrency,cryptocurrency mining revenue,"digital assets, net",digital asset purchases,digital asset impairment losses,revenue recognized from cryptocurrencies mined,
1,"revenue, cryptocurrency mining, net","revenue, cryptocurrency mining",impairment of mined cryptocurrency,bitcoin\n investment trust,"digital assets gain, net",digital asset impairment losses,digital asset impairment losses,purchase of miner equipment with cryptocurrencies,None


### Export to CSV

In [15]:
#data_location_table.to_csv('final_results_table_location.csv', index=False)

## Results for Count of Keywords and Count of Keywords Within Table

In [16]:
results = company_df.merge(data,on='URL')
results

,CIK,Company Name,SIC,Industry,State Location,State Inc,Filling Date,URL,digital asset,cryptocurrency,cryptocurrencies,digital holdings,bitcoin,ethereum,digital asset_table,cryptocurrency_table,cryptocurrencies_table,digital holdings_table,bitcoin_table,ethereum_table
0,0000896493,"BitNile Holdings, Inc.",3679,ELECTRONIC COMPONENTS,NV,DE,2022-04-15,https://www.sec.gov/Archives/edgar/data/896493...,76,224,122,2,297,10,0,11,4,2,3,0
1,0000896493,"BitNile Holdings, Inc.",3679,ELECTRONIC COMPONENTS,NV,DE,2021-04-15,https://www.sec.gov/Archives/edgar/data/896493...,4,17,4,0,5,1,0,5,0,0,0,0
2,0001507605,"MARATHON DIGITAL HOLDINGS, INC.",7374,SERVICES,NV,NV,2022-03-10,https://www.sec.gov/Archives/edgar/data/150760...,208,49,38,73,387,0,4,5,4,1,5,0
3,0001507605,"MARATHON DIGITAL HOLDINGS, INC.",7374,SERVICES,NV,NV,2021-03-16,https://www.sec.gov/Archives/edgar/data/150760...,201,33,17,45,294,0,4,1,0,1,5,0
4,0001318605,"Tesla, Inc.",3711,MOTOR VEHICLES & PASSENGER CAR BODIES,CA,DE,2022-02-07,https://www.sec.gov/Archives/edgar/data/131860...,49,0,0,0,12,0,4,0,0,0,0,0
5,0001318605,"Tesla, Inc.",3711,MOTOR VEHICLES & PASSENGER CAR BODIES,CA,DE,2021-02-08,https://www.sec.gov/Archives/edgar/data/131860...,17,0,0,0,9,0,0,0,0,0,0,0
6,0001050446,MICROSTRATEGY Inc,7372,SERVICES,VA,DE,2022-02-16,https://www.sec.gov/Archives/edgar/data/105044...,105,5,7,0,516,0,12,0,0,0,25,0
7,0001050446,MICROSTRATEGY Inc,7372,SERVICES,VA,DE,2021-02-12,https://www.sec.gov/Archives/edgar/data/105044...,86,3,5,0,353,0,9,0,0,0,5,0
8,0001167419,"Riot Blockchain, Inc.",7374,SERVICES,CO,NV,2022-03-16,https://www.sec.gov/Archives/edgar/data/116741...,8,72,164,0,321,0,0,0,26,0,2,0
9,0001167419,"Riot Blockchain, Inc.",7374,SERVICES,CO,NV,2021-03-31,https://www.sec.gov/Archives/edgar/data/116741...,11,225,241,2,245,0,0,1,2,0,0,0


### Export to CSV

In [17]:
#results.to_csv('final_results.csv', index=False)

In [18]:
print('RUN TIME: ',datetime.now() - start_time)

RUN TIME:  0:00:04.463094
